In [1]:
import numpy as np
import pandas as pd
from helper import (
    load_dataset,
    split_data,
    run_single_experiment,
    run_all_experiments,
    tune_hyperparams
)

from cnn_model import cnn_model
from loss_functions import (
    symmetric_cross_entropy,
    forward_correction_loss,
    CoTeachingProxyLoss,
    RememberRateScheduler,
    _infer_noise_rate_from_name, 
)
import anchor_estimator
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

import importlib


In [ ]:
datasets = ['FashionMNIST0.3', 'FashionMNIST0.6', 'CIFAR']
methods  = ['sce']

# One-time tuning per dataset × method
tuned = {}  # key: (dataset, method) -> param dict
for dataset in datasets:
    data_path = f'datasets/{dataset}.npz'
    Xtr, Str, Xts, Yts, T = load_dataset(data_path, dataset)
    input_shape = Xtr.shape[1:]

    for method in methods:
        print(f"\n=== Tuning {method.upper()} on {dataset} ===")
        best_params = tune_hyperparams(
            Xtr, Str, dataset, method, input_shape,
            n_dev_runs=3, epochs=30  # lightweight dev budget
        )
        tuned[(dataset, method)] = best_params


=== Tuning SCE on FashionMNIST0.3 ===


2025-11-04 18:26:54.228285: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4 Max
2025-11-04 18:26:54.228309: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 36.00 GB
2025-11-04 18:26:54.228313: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 13.50 GB
I0000 00:00:1762241214.228328 16171413 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1762241214.228351 16171413 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2025-11-04 18:26:54.594201: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


[TUNE] dataset=FashionMNIST0.3 method=sce params={'alpha': 0.01, 'beta': 0.5, 'A': -1.0, 'lr': 0.001} -> val_acc=0.6811
[TUNE] dataset=FashionMNIST0.3 method=sce params={'alpha': 0.01, 'beta': 0.5, 'A': -2.0, 'lr': 0.001} -> val_acc=0.6807
[TUNE] dataset=FashionMNIST0.3 method=sce params={'alpha': 0.01, 'beta': 0.5, 'A': -4.0, 'lr': 0.001} -> val_acc=0.6808
[TUNE] dataset=FashionMNIST0.3 method=sce params={'alpha': 0.01, 'beta': 1.0, 'A': -1.0, 'lr': 0.001} -> val_acc=0.6819
[TUNE] dataset=FashionMNIST0.3 method=sce params={'alpha': 0.01, 'beta': 1.0, 'A': -2.0, 'lr': 0.001} -> val_acc=0.6805
[TUNE] dataset=FashionMNIST0.3 method=sce params={'alpha': 0.01, 'beta': 1.0, 'A': -4.0, 'lr': 0.001} -> val_acc=0.6738
[TUNE] dataset=FashionMNIST0.3 method=sce params={'alpha': 0.05, 'beta': 0.5, 'A': -1.0, 'lr': 0.001} -> val_acc=0.6776
[TUNE] dataset=FashionMNIST0.3 method=sce params={'alpha': 0.05, 'beta': 0.5, 'A': -2.0, 'lr': 0.001} -> val_acc=0.6806
[TUNE] dataset=FashionMNIST0.3 method=sc

In [ ]:
# hyperparams for sce
tuned

{('FashionMNIST0.3', 'sce'): {'alpha': 0.01,
  'beta': 1.0,
  'A': -1.0,
  'lr': 0.001},
 ('FashionMNIST0.6', 'sce'): {'alpha': 0.01,
  'beta': 0.5,
  'A': -4.0,
  'lr': 0.001},
 ('CIFAR', 'sce'): {'alpha': 0.05, 'beta': 1.0, 'A': -4.0, 'lr': 0.001}}

In [4]:
datasets = ['FashionMNIST0.3', 'FashionMNIST0.6', 'CIFAR']
methods  = ['coteaching']

# One-time tuning per dataset × method
tuned_coteaching = {}  # key: (dataset, method) -> param dict
for dataset in datasets:
    data_path = f'datasets/{dataset}.npz'
    Xtr, Str, Xts, Yts, T = load_dataset(data_path, dataset)
    input_shape = Xtr.shape[1:]

    for method in methods:
        print(f"\n=== Tuning {method.upper()} on {dataset} ===")
        best_params = tune_hyperparams(
            Xtr, Str, dataset, method, input_shape,
            n_dev_runs=3, epochs=30  # lightweight dev budget
        )
        tuned_coteaching[(dataset, method)] = best_params


=== Tuning COTEACHING on FashionMNIST0.3 ===
[TUNE] dataset=FashionMNIST0.3 method=coteaching params={'warmup': 3, 'lr': 0.001} -> val_acc=0.6779
[TUNE] dataset=FashionMNIST0.3 method=coteaching params={'warmup': 3, 'lr': 0.0005} -> val_acc=0.6778
[TUNE] dataset=FashionMNIST0.3 method=coteaching params={'warmup': 5, 'lr': 0.001} -> val_acc=0.6801
[TUNE] dataset=FashionMNIST0.3 method=coteaching params={'warmup': 5, 'lr': 0.0005} -> val_acc=0.6784
[TUNE] dataset=FashionMNIST0.3 method=coteaching params={'warmup': 10, 'lr': 0.001} -> val_acc=0.6811
[TUNE] dataset=FashionMNIST0.3 method=coteaching params={'warmup': 10, 'lr': 0.0005} -> val_acc=0.6756
[TUNE][BEST] dataset=FashionMNIST0.3 method=coteaching -> {'warmup': 10, 'lr': 0.001} (val_acc=0.6811)

=== Tuning COTEACHING on FashionMNIST0.6 ===
[TUNE] dataset=FashionMNIST0.6 method=coteaching params={'warmup': 3, 'lr': 0.001} -> val_acc=0.3821
[TUNE] dataset=FashionMNIST0.6 method=coteaching params={'warmup': 3, 'lr': 0.0005} -> val_ac

In [5]:
tuned_coteaching

{('FashionMNIST0.3', 'coteaching'): {'warmup': 10, 'lr': 0.001},
 ('FashionMNIST0.6', 'coteaching'): {'warmup': 5, 'lr': 0.0005},
 ('CIFAR', 'coteaching'): {'warmup': 5, 'lr': 0.0005}}

In [3]:
datasets = ['FashionMNIST0.3', 'FashionMNIST0.6', 'CIFAR']
methods = ['forward', 'sce', 'coteaching'] 

result = run_all_experiments(datasets, methods, 10, 50)

In [26]:
pivot_df = result.pivot(index='Dataset', columns='Method', values='Result')
    
print(pivot_df)

Method                    SCE
Dataset                      
CIFAR            65.57 ± 3.22
FashionMNIST0.3  98.58 ± 0.16
FashionMNIST0.6  95.83 ± 0.63
